In [1]:
import pickle
import sys
sys.path.append("..")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from nsga2.examples.fifa_ultimate_team.crossover import SquadCrossover
from nsga2.examples.fifa_ultimate_team.genotype import Squad
from nsga2.examples.fifa_ultimate_team.initializer import SquadInitializer
from nsga2.examples.fifa_ultimate_team.mutator import SquadMutator
from nsga2.examples.fifa_ultimate_team.stopper import SquadStopper
from nsga2.nsga2.evaluator import Evaluator
from nsga2.nsga2.nsga2 import NSGA2
from nsga2.nsga2.selector import TournamentSelector
from nsga2.nsga2.stopper import MaxGenerationStopper

In [2]:
with open("../data/processed/players.p", "rb") as file_pointer:
    players = pickle.load(file_pointer)

In [3]:
with open("../data/processed/formations.p", "rb") as file_pointer:
    formations = pickle.load(file_pointer)

ModuleNotFoundError: No module named 'fut_squad_evolver'

In [ ]:
players = players[(players["price"] > 0) & (players["metal"] == "Gold") & (players["revision"] == "Normal")]
formation = formations["541"]
greater_is_better_dict = {"price": False, "overall": True, "chemistry": True}

In [ ]:
formation.labels

In [ ]:
locked_players = {2: 195, 10: 271}

In [ ]:
initializer = SquadInitializer(1000, formation, players, 2, locked_players=locked_players)
evaluator = Evaluator(non_dominated_sorter_kwargs={"greater_is_better_dict": greater_is_better_dict, "max_n_obs": 100}, crowding_distance_sorter_kwargs={"greater_is_better_dict": greater_is_better_dict})
selector = TournamentSelector()
crossover = SquadCrossover(0.2)
mutator = SquadMutator(players, 2, 0.75, 0.7, 0.0, 0.3)
stopper = SquadStopper(max_price=60000, min_overall=82, min_chemistry=90)

nsga2 = NSGA2(initializer, evaluator, selector, crossover, mutator, stopper)

In [ ]:
population = nsga2.search()

In [ ]:
len(nsga2.population_log)

In [ ]:
eval_df = []
for i, pop in enumerate(nsga2.population_log):
    for individual in pop.values():
        temp_dict = individual.phenotype
        temp_dict["pareto_front"] = individual.fitness["pareto_front"] if individual.fitness != {} else None
        temp_dict["generation"] = i
        eval_df.append(temp_dict)
eval_df = pd.DataFrame(eval_df)
eval_df["performance_ratio"] = eval_df["overall"] * eval_df["chemistry"] / eval_df["price"]

In [ ]:
evaluation = eval_df.groupby("generation").agg({"price": ["min", "mean", "max"], "overall": ["min", "mean", "max"], "chemistry": ["min", "mean", "max"], "performance_ratio": ["min", "mean", "max"]})

In [ ]:
fig = plt.figure()
ax = fig.gca()

evaluation["performance_ratio"].plot(ax=ax)
ax.set_ylim(0, 1)

In [ ]:
evaluation_0 = eval_df[eval_df["pareto_front"] == 0].groupby("generation").agg({"price": ["min", "mean", "max"], "overall": ["min", "mean", "max"], "chemistry": ["min", "mean", "max"]})

In [ ]:
evaluation_0["overall"].plot()

In [ ]:
pd.DataFrame([individual.phenotype for individual in population.values() if 
              (individual.fitness != {} and individual.fitness["pareto_front"] == 0)\
              and individual.phenotype["overall"] >= 80 \
              and individual.phenotype["chemistry"] >= 90]).sort_values("price").head(40)

In [ ]:
my_genotype = [individual for individual in population.values() if individual.individual_id == 41457][0].genotype

In [ ]:
print(my_genotype)